# Favorita Grocery Sales Forecasting

## 1. Introduction
This notebook covers the end-to-end pipeline for forecasting grocery sales using the Favorita dataset. 
We will perform:
- Data Loading & Preprocessing
- Feature Engineering (Lags, Rolling Stats, Date features)
- Model Training (LightGBM, XGBoost, Random Forest)
- Evaluation & Submission

## 2. Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import warnings
import os

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Check working directory and available files
print("Current Working Directory:", os.getcwd())
print("Files in Directory:", os.listdir())

## 3. Data Loading
Loading the datasets: `train`, `test`, `items`, `stores`, `oil`, `holidays_events`.
Note: Ensure these files are present in the directory.

In [ ]:
def load_data():
    files = ['train.csv', 'test.csv', 'items.csv', 'stores.csv', 'oil.csv', 'holidays_events.csv']
    missing_files = [f for f in files if f not in os.listdir()]
    
    if missing_files:
        print(f"Warning: The following files are missing: {missing_files}")
        print("Please upload them to the directory. Assuming mock/placeholder behavior for now if needed.")
        # Return None or handle gracefully if you want to proceed with mock data (optional)
        # For now, we will attempt to load what exists or fail gracefully.
    
    data = {}
    try:
        # Define types to save memory
        dtypes = {
            'id': 'int32',
            'item_nbr': 'int32',
            'store_nbr': 'int8',
            'unit_sales': 'float32',
            'onpromotion': 'object'
        }
        
        if 'train.csv' in os.listdir():
            print("Loading train.csv...")
            # Reading a subset for development if file is huge; remove nrows=1000000 for full training
            data['train'] = pd.read_csv('train.csv', dtype=dtypes, parse_dates=['date'], nrows=1000000)
        
        if 'test.csv' in os.listdir():
            print("Loading test.csv...")
            data['test'] = pd.read_csv('test.csv', dtype=dtypes, parse_dates=['date'])
            
        if 'items.csv' in os.listdir():
            data['items'] = pd.read_csv('items.csv')
            
        if 'stores.csv' in os.listdir():
            data['stores'] = pd.read_csv('stores.csv')
            
        if 'oil.csv' in os.listdir():
            data['oil'] = pd.read_csv('oil.csv', parse_dates=['date'])
            
        if 'holidays_events.csv' in os.listdir():
            data['holidays'] = pd.read_csv('holidays_events.csv', parse_dates=['date'])
            
        print("Data loading complete.")
        return data
    except Exception as e:
        print(f"Error loading data: {e}")
        return {}

data_dict = load_data()

## 4. Preprocessing
- Merge with metadata (items, stores)
- Handle missing dates in oil data
- Process holidays

In [ ]:
def preprocess_data(data):
    if 'train' not in data:
        print("Train data not found. Skipping preprocessing.")
        return None, None
    
    train = data['train']
    test = data.get('test')
    items = data.get('items')
    stores = data.get('stores')
    oil = data.get('oil')
    holidays = data.get('holidays')
    
    # 1. Merge Oil Data (Interpolate missing values)
    if oil is not None:
        # Create full date range for oil to handle gaps
        date_range = pd.date_range(start=oil['date'].min(), end=oil['date'].max())
        oil = oil.set_index('date').reindex(date_range).reset_index()
        oil.rename(columns={'index': 'date', 'dcoilwtico': 'oil_price'}, inplace=True)
        # Interpolate missing oil prices
        oil['oil_price'] = oil['oil_price'].interpolate(method='linear').fillna(method='bfill')
        
        train = train.merge(oil, on='date', how='left')
        if test is not None:
            test = test.merge(oil, on='date', how='left')
            
    # 2. Merge Stores and Items
    if stores is not None:
        train = train.merge(stores, on='store_nbr', how='left')
        if test is not None:
            test = test.merge(stores, on='store_nbr', how='left')
            
    if items is not None:
        train = train.merge(items, on='item_nbr', how='left')
        if test is not None:
            test = test.merge(items, on='item_nbr', how='left')
            
    # 3. Handle Holidays (Simplified for now: is_holiday flag)
    if holidays is not None:
        holidays = holidays[holidays['transferred'] == False]
        holiday_dates = set(holidays['date'])
        train['is_holiday'] = train['date'].apply(lambda x: 1 if x in holiday_dates else 0)
        if test is not None:
            test['is_holiday'] = test['date'].apply(lambda x: 1 if x in holiday_dates else 0)
            
    # 4. Fill Missing Values
    # Onpromotion often has NaNs, fill with False (or 0)
    train['onpromotion'] = train['onpromotion'].fillna(False).astype(bool)
    if test is not None:
        test['onpromotion'] = test['onpromotion'].fillna(False).astype(bool)
        
    print("Preprocessing complete. Train shape:", train.shape)
    return train, test

train_df, test_df = preprocess_data(data_dict)

## 5. Feature Engineering
- Date features (day, month, year, dayofweek)
- Lag features (sales lags)
- Rolling features

In [ ]:
def feature_engineering(df):
    if df is None:
        return None
    
    print("Starting feature engineering...")
    df['date'] = pd.to_datetime(df['date'])
    
    # Date Features
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
    df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
    
    # NOTE: Lags and Rolling stats are tricky with simplified loading.
    # We need a continuous time series for each store-item combination.
    # For this baseline, we will perform a simple sort and shift assumption or skip complex lags if memory is tight.
    
    # Sorting
    df = df.sort_values(by=['store_nbr', 'item_nbr', 'date'])
    
    # Simple Lag Features (lag 7 days, lag 14 days)
    # Groupby is expensive on large data. Use carefully.
    # For demonstration, we will just take a few lags
    # df['sales_lag_7'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].shift(7)
    # df['sales_lag_14'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].shift(14)
    
    # Filling lag NaNs with 0 or mean
    # df.fillna(0, inplace=True)
    
    print("Feature engineering complete.")
    return df

train_df = feature_engineering(train_df)
if test_df is not None:
    test_df = feature_engineering(test_df)

## 6. Model Training
We will try:
1. Random Forest (Baseline)
2. LightGBM
3. XGBoost

Metric: RMSE & MAE

In [ ]:
def train_models(df):
    if df is None:
        return None, None
    
    # Exclude non-numeric or leak columns
    drop_cols = ['id', 'date', 'unit_sales', 'description', 'locale', 'locale_name', 'type_x', 'type_y', 'city', 'state']
    # Encode categorical columns if any remain
    cat_cols = ['family', 'type', 'city', 'state', 'description'] # Check overlap with drop_cols
    
    # Simple encoding for object columns
    for col in df.select_dtypes(include=['object']).columns:
        if col not in drop_cols:
            le = LabelEncoder()
            df[col] = df[col].astype(str)
            df[col] = le.fit_transform(df[col])
            
    # Define Features and Target
    features = [c for c in df.columns if c not in drop_cols and c != 'unit_sales']
    target = 'unit_sales'
    
    # Train/Validation Split (Time-based)
    # Use last month of train data as validation
    val_date_start = df['date'].max() - pd.Timedelta(days=28)
    
    X_train = df[df['date'] < val_date_start][features]
    y_train = df[df['date'] < val_date_start][target]
    X_val = df[df['date'] >= val_date_start][features]
    y_val = df[df['date'] >= val_date_start][target]
    
    # Handle NaNs in X (Tree models handle them or we impute)
    X_train = X_train.fillna(0)
    X_val = X_val.fillna(0)
    
    # --- Random Forest ---
    print("Training Random Forest...")
    rf = RandomForestRegressor(n_estimators=50, max_depth=10, n_jobs=-1, random_state=42)
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_val)
    rf_rmse = np.sqrt(mean_squared_error(y_val, rf_pred))
    print(f"Random Forest RMSE: {rf_rmse:.4f}")
    
    # --- LightGBM ---
    print("Training LightGBM...")
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9
    }
    gbm = lgb.train(params, lgb_train, num_boost_round=1000, valid_sets=[lgb_val], callbacks=[lgb.early_stopping(stopping_rounds=50)])
    
    # --- XGBoost ---
    print("Training XGBoost...")
    xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.05, max_depth=6)
    xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False)
    xgb_pred = xgb_reg.predict(X_val)
    xgb_rmse = np.sqrt(mean_squared_error(y_val, xgb_pred))
    print(f"XGBoost RMSE: {xgb_rmse:.4f}")
    
    return gbm, features

## 7. Submission Generation
Generate predictions for the test set and create `submission.csv`.

In [ ]:
# Train models (assuming train_df is available from previous cells)
if train_df is not None:
    model, features = train_models(train_df)
    
    # Save the model for the app
    with open('model.pkl', 'wb') as f:
        pickle.dump(model, f)
    print("Model saved to model.pkl")
else:
    print("Train data missing. Cannot train model.")
    model = None

# Predictions for Submission
if model is not None and test_df is not None:
    print("Generating predictions for test set...")
    
    # Ensure test features match train features (preprocessing handled strings similarly)
    # Note: In production, we'd need to apply the exact same LabelEncoders mappings.
    # For this baseline, we relied on simple string conversion/encoding. 
    # Ensure standard LabelEncoding (re-fit on test) is avoided. Ideally, fit on train, transform on test.
    # Here we will just ensure columns match.
    
    # Re-apply simplified encoding logic for test (mocking consistent pipeline)
    drop_cols = ['id', 'date', 'unit_sales', 'description', 'locale', 'locale_name', 'type_x', 'type_y', 'city', 'state']
    for col in test_df.select_dtypes(include=['object']).columns:
        if col not in drop_cols:
             le = LabelEncoder()
             test_df[col] = test_df[col].astype(str)
             # Warning: This is a hack for baseline. Unseen labels will cause issues if not handled.
             test_df[col] = le.fit_transform(test_df[col])

    X_test = test_df[features]
    X_test = X_test.fillna(0)
    
    preds = model.predict(X_test)
    
    submission = pd.DataFrame({
        'id': test_df['id'],
        'unit_sales': preds
    })
    
    # Negative sales could be clipped to 0
    submission['unit_sales'] = submission['unit_sales'].clip(lower=0)
    
    submission.to_csv('submission.csv', index=False)
    print("submission.csv created successfully.")
else:
    print("Cannot generate submission. Missing model or test data.")